In [1]:
import itertools

import torch

from torch_geometric.nn import GCNConv
from torch_geometric.nn.pool import global_mean_pool
from torch.utils.data import DataLoader, random_split

In [2]:
from helpers import CVFConfigForGCNDataset

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
color_mapping_categories = 15

# dataset = CVFConfigDataset(
#     "small_graph_test_config_rank_dataset.csv", "small_graph_edge_index.json", 4
# )

# dataset_n1 = CVFConfigForGCNDataset(
#     device,
#     "graph_1_config_rank_dataset.csv",
#     "graph_1_edge_index.json",
#     20
# )

# dataset_n4 = CVFConfigForGCNDataset(
#     device,
#     "graph_4_config_rank_dataset.csv",
#     "graph_4_edge_index.json",
#     color_mapping_categories
# )

# dataset_n5 = CVFConfigForGCNDataset(
#     device,
#     "graph_5_config_rank_dataset.csv",
#     "graph_5_edge_index.json",
#     color_mapping_categories
# )

# dataset_n6 = CVFConfigForGCNDataset(
#     device,
#     "graph_6_config_rank_dataset.csv",
#     "graph_6_edge_index.json",
#     color_mapping_categories,
# )

# dataset_n6b = CVFConfigForGCNDataset(
#     device,
#     "graph_6b_config_rank_dataset.csv",
#     "graph_6b_edge_index.json",
#     color_mapping_categories,
# )

# dataset_n7 = CVFConfigForGCNDataset(
#     device,
#     "graph_7_config_rank_dataset.csv",
#     "graph_7_edge_index.json",
#     color_mapping_categories,
# )

# # dataset_n8 = CVFConfigForGCNDataset(
# #     device,
# #     "graph_8_config_rank_dataset.csv",
# #     "graph_8_edge_index.json",
# #     color_mapping_categories,
# # )

# dataset_pl_n5 = CVFConfigForGCNDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n5_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n5_edge_index.json",
#     color_mapping_categories
# )

# dataset_pl_n6 = CVFConfigForGCNDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n6_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n6_edge_index.json",
#     color_mapping_categories
# )

# dataset_pl_n7 = CVFConfigForGCNDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n7_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n7_edge_index.json",
#     color_mapping_categories
# )

# dataset_pl_n8 = CVFConfigForGCNDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n8_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n8_edge_index.json",
#     color_mapping_categories
# )

# dataset_pl_n9 = CVFConfigForGCNDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n9_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n9_edge_index.json",
#     color_mapping_categories,
# )

# dataset_pl_n12 = CVFConfigForGCNDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n12_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n12_edge_index.json",
#     color_mapping_categories
# )

# dataset_implicit_n15 = CVFConfigForGCNDataset(
#     device,
#     "implicit_graph_n15_config_rank_dataset.csv",
#     "implicit_graph_n15_edge_index.json",
#     color_mapping_categories,
#     one_hot_encode=False
# )

# dataset_implicit_n10 = CVFConfigForGCNDataset(
#     device,
#     "implicit_graph_n10_config_rank_dataset.csv",
#     "implicit_graph_n10_edge_index.json",
#     color_mapping_categories,
#     one_hot_encode=True
# )

# dataset_implicit_n4 = CVFConfigForGCNDataset(
#     device,
#     "implicit_graph_n4_config_rank_dataset.csv",
#     "implicit_graph_n4_edge_index.json",
#     color_mapping_categories,
#     one_hot_encode=True
# )

# dataset_implicit_n5 = CVFConfigForGCNDataset(
#     device,
#     "implicit_graph_n5_config_rank_dataset.csv",
#     "implicit_graph_n5_edge_index.json",
# )

dataset_rr_n4 = CVFConfigForGCNDataset(
    device,
    "graph_random_regular_graph_n4_d3_config_rank_dataset.csv",
    "graph_random_regular_graph_n4_d3_edge_index.json",
)

dataset_rr_n5 = CVFConfigForGCNDataset(
    device,
    "graph_random_regular_graph_n5_d4_config_rank_dataset.csv",
    "graph_random_regular_graph_n5_d4_edge_index.json",
)

dataset_rr_n6 = CVFConfigForGCNDataset(
    device,
    "graph_random_regular_graph_n6_d3_config_rank_dataset.csv",
    "graph_random_regular_graph_n6_d3_edge_index.json",
)

dataset_rr_n7 = CVFConfigForGCNDataset(
    device,
    "graph_random_regular_graph_n7_d4_config_rank_dataset.csv",
    "graph_random_regular_graph_n7_d4_edge_index.json",
)

dataset_rr_n8 = CVFConfigForGCNDataset(
    device,
    "graph_random_regular_graph_n8_d4_config_rank_dataset.csv",
    "graph_random_regular_graph_n8_d4_edge_index.json",
)


dataset_tiny_test = CVFConfigForGCNDataset(
    device,
    "tiny_graph_test_config_rank_dataset.csv",
    "tiny_graph_edge_index.json",
)

dataset_graph_1 = CVFConfigForGCNDataset(
    device,
    "graph_1_config_rank_dataset.csv",
    "graph_1_edge_index.json",
)


batch_size = 512

dataset_coll = [
    dataset_rr_n8
]

train_dataloader_coll = []
test_dataloader_coll = []

for dataset in dataset_coll:
    train_size = int(0.9 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    train_dataloader_coll.append(train_loader)
    test_dataloader_coll.append(test_loader)

train_dataloader_coll_iter = [iter(i) for i in train_dataloader_coll]

In [5]:
def generate_batch():
    end_loop = [False for _ in range(len(train_dataloader_coll))]
    while not any(end_loop):
        for di, data_loader in enumerate(train_dataloader_coll_iter):
            if end_loop[di]:
                continue
            try:
                batch = next(data_loader)
            except StopIteration:
                end_loop[di] = True
                continue
            yield batch, di

In [6]:
print("Number of batches:", [len(i) for i in train_dataloader_coll])

Number of batches: [687]


In [7]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, edge_index):
        h = self.gcn1(x, edge_index)
        h = torch.relu(h)
        h = self.gcn2(h, edge_index)
        h = torch.relu(h)
        h = self.out(h)
        h = torch.relu(h)
        h = global_mean_pool(h, torch.zeros(h.size(1)).to(device).long())
        return h

    def fit(self, epochs):
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=0.01)
        # edge_index = dataset.edge_index.t().to(device)
        dataloaders = itertools.tee(generate_batch(), epochs)
        for epoch in range(1, epochs + 1):
            self.train()
            total_loss = 0
            count = 0
            for batch, di in dataloaders[epoch - 1]:
                x = batch[0]
                y = batch[1]
                y = y.unsqueeze(-1)
                optimizer.zero_grad()
                out = self(x, dataset_coll[di].edge_index)
                # print(out.shape, y.shape)
                loss = criterion(out, y)
                total_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            if count > 0:
                print(
                    "Training set | Epoch",
                    epoch,
                    "| Loss:",
                    (total_loss / count).item(),
                )

In [8]:
gnn = VanillaGNN(1, 64, 1).to(device)
print(gnn)

gnn.fit(epochs=250)

VanillaGNN(
  (gcn1): GCNConv(1, 64)
  (gcn2): GCNConv(64, 64)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
Training set | Epoch 1 | Loss: 1.3277958631515503
Training set | Epoch 2 | Loss: 1.2286453247070312
Training set | Epoch 3 | Loss: 1.2268195152282715
Training set | Epoch 4 | Loss: 1.2256735563278198
Training set | Epoch 5 | Loss: 1.2250410318374634
Training set | Epoch 6 | Loss: 1.2241405248641968
Training set | Epoch 7 | Loss: 1.2238231897354126
Training set | Epoch 8 | Loss: 1.2232012748718262
Training set | Epoch 9 | Loss: 1.2228171825408936
Training set | Epoch 10 | Loss: 1.2221161127090454
Training set | Epoch 11 | Loss: 1.222428798675537
Training set | Epoch 12 | Loss: 1.2217425107955933
Training set | Epoch 13 | Loss: 1.221452236175537
Training set | Epoch 14 | Loss: 1.2207239866256714
Training set | Epoch 15 | Loss: 1.2206833362579346
Training set | Epoch 16 | Loss: 1.2205160856246948
Training set | Epoch 17 | Loss: 1.2203260660171509
Training set | Epoch

In [9]:
# testing
import csv

torch.no_grad()
torch.set_printoptions(profile="full")

f = open("test_result.csv", "w", newline="")
csv_writer = csv.writer(f)
csv_writer.writerow(["Actual", "Predicted"])

criterion = torch.nn.MSELoss()
total_loss = 0
total_matched = 0

indx = 0
dataset = dataset_coll[indx]
test_loader = test_dataloader_coll[indx]
test_dataset = test_loader.dataset

count = 0
for batch in test_loader:
    x = batch[0]
    y = batch[1]
    y = y.unsqueeze(-1)
    out = gnn(x, dataset.edge_index)
    csv_writer.writerows(zip(y.detach().cpu().numpy(), out.detach().cpu().numpy()))
    loss = criterion(out, y)
    total_loss += loss
    out = torch.round(out)
    matched = (out == y).sum().item()
    total_matched += matched
    count += 1

f.close()

print(
    "Test loss:",
    total_loss.detach() / count,
    "Total matched",
    total_matched,
    "out of",
    len(test_dataset),
    f"({round(total_matched/len(test_dataset) * 100, 2)}%)",
)

Test loss: tensor(1.2060, device='cuda:0') Total matched 13165 out of 39063 (33.7%)
